**Objective:**
* Merge hospital and payer dataset
* Detect discrepancies in pricing columns for observation level hospital, payer, billing code
* Root cause of discrepancies and pick one single rate for each `ein`

Date: 3/6/25 <br>
Liuna Issagholian, data sceintist 

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
# load datasets as dataframes
#hospital dataset
dfh = pd.read_csv('hpt_extract_20250213.csv')

# payer dataset
dfp = pd.read_csv('tic_extract_20250213.csv')

print('Shape of hospital dataset: ' + str(dfh.shape))
print('Shape of payer dataset: ' + str(dfp.shape))

Shape of hospital dataset: (2950, 22)
Shape of payer dataset: (222, 17)


In [3]:
# quick overview of datasets for checking column datatypes and missing values
dfh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2950 entries, 0 to 2949
Data columns (total 22 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   source_file_name                       2950 non-null   object 
 1   hospital_id                            2950 non-null   object 
 2   hospital_name                          2950 non-null   object 
 3   last_updated_on                        2950 non-null   object 
 4   hospital_state                         2950 non-null   object 
 5   license_number                         2950 non-null   object 
 6   payer_name                             2950 non-null   object 
 7   plan_name                              2950 non-null   object 
 8   code_type                              2950 non-null   object 
 9   raw_code                               2950 non-null   object 
 10  description                            2950 non-null   object 
 11  sett

In [4]:
dfp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   payer                       222 non-null    object 
 1   network_name                222 non-null    object 
 2   network_id                  222 non-null    object 
 3   network_year_month          222 non-null    int64  
 4   network_region              222 non-null    object 
 5   code                        222 non-null    int64  
 6   code_type                   222 non-null    object 
 7   ein                         222 non-null    int64  
 8   taxonomy_filtered_npi_list  218 non-null    object 
 9   modifier_list               7 non-null      object 
 10  billing_class               222 non-null    object 
 11  place_of_service_list       197 non-null    object 
 12  negotiation_type            222 non-null    object 
 13  arrangement                 222 non

Both datasets have some missing values.

In [5]:
dfh.head()

,source_file_name,hospital_id,hospital_name,last_updated_on,hospital_state,license_number,payer_name,plan_name,code_type,raw_code,description,setting,modifiers,standard_charge_gross,standard_charge_discounted_cash,standard_charge_negotiated_dollar,standard_charge_negotiated_percentage,standard_charge_min,standard_charge_max,standard_charge_methodology,additional_payer_notes,additional_generic_notes
0,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Aetna,Medicare,CPT,99283,EMERGENCY DEPT VISIT LOW MDM,outpatient,NaN,NaN,NaN,323.34,NaN,83.78,1009.22,fee schedule,NaN,NaN
1,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,HealthFirst,Commercial Enrollees,CPT,43239,EGD BIOPSY SINGLE/MULTIPLE,outpatient,NaN,NaN,NaN,1037.65,NaN,165.40,3206.34,fee schedule,NaN,NaN
2,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Aetna,Commercial,CPT,43239,UPPER GI ENDOSCOPY BIOPSY,outpatient,NaN,NaN,NaN,1246.73,NaN,1246.73,1394.79,fee schedule,NaN,NaN
3,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Cigna,LocalPlus,CPT,99283,HC EMERGENCY DEPT VISIT LVL 3,outpatient,NaN,3744.0,2433.6,1797.00,NaN,225.00,1797.00,other,NaN,per visit
4,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Oscar,Medicare,CPT,43239,EGD BIOPSY SINGLE/MULTIPLE,outpatient,NaN,NaN,NaN,1037.65,NaN,141.77,1815.10,fee schedule,NaN,NaN


In [6]:
dfp.head()

,payer,network_name,network_id,network_year_month,network_region,code,code_type,ein,taxonomy_filtered_npi_list,modifier_list,billing_class,place_of_service_list,negotiation_type,arrangement,rate,cms_baseline_schedule,cms_baseline_rate
0,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,872,MS-DRG,131740114,"1003990763,1023202793,1063525152,1063606739,10...",NaN,institutional,NaN,negotiated,ffs,15902.00,IPPS,6829.75
1,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,99283,CPT,131624096,"1003255670,1245759711,1487026522,1598095267,16...",NaN,professional,11,negotiated,ffs,123.86,PFS_NONFACILITY_1320201,76.89
2,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,43239,CPT,131740114,"1700348620,1700892056,1922539964,1942685292",NaN,professional,11,negotiated,ffs,993.92,PFS_NONFACILITY_1320202,424.76
3,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,872,MS-DRG,133971298,"1245635200,1437523537,1528013695,1528432622,15...",NaN,institutional,NaN,negotiated,ffs,27924.63,IPPS,6829.75
4,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,43239,CPT,131740114,1346697315,NaN,professional,11,negotiated,ffs,849.63,PFS_NONFACILITY_1320203,391.85


In [7]:
# make lower case for all hospitals and pay
def lower_case(df):
    df[df.select_dtypes(include='object').columns] = df.select_dtypes(include='object').apply(lambda col: col.str.lower())
    return df

dfh, dfp = lower_case(dfh) , lower_case(dfp)

In [8]:
# check for dupicated rows:
print('number of duplicated rows in hospital dataset: ' + str(len(dfh[dfh.duplicated()])))

print('number of duplicated rows in payer dataset: ' + str(len(dfp[dfp.duplicated()])))

number of duplicated rows in hospital dataset: 3
number of duplicated rows in payer dataset: 0


In [9]:
#dropping duplicates from hospital datasets
dfh = dfh.drop_duplicates().reset_index(drop=True)

print('after dropping duplication:')
dfh.shape

after dropping duplication:


(2947, 22)

In [10]:
# checking columns with only value
# Print column names with one unique value
print("Columns for hospital with one unique value:", list(dfh.columns[dfh.nunique() == 1]))
print("Columns for payer with one unique value:", list(dfp.columns[dfp.nunique() == 1]))

Columns for hospital with one unique value: ['hospital_state', 'modifiers']
Columns for payer with one unique value: ['network_year_month', 'network_region', 'arrangement']


In [11]:
def del_single_values(df):
    df = df.drop(list(df.columns[df.nunique() == 1]), axis=1)
    return df
    
dfh, dfp= del_single_values(dfh), del_single_values(dfp)

In [12]:
dfh.shape, dfp.shape

((2947, 20), (222, 14))

In [13]:
dfh.source_file_name.value_counts()

source_file_name
133971298-1801992631_nyu-langone-tisch_standardcharges.csv.gz    2322
13-1740114_montefiore-medical-center_standardcharges.csv.gz       377
131624096_mount-sinai-hospital_standardcharges.csv.gz             248
Name: count, dtype: int64

In [14]:
#check to see these are all conveying same info
h_dup = dfh[['hospital_id', 'hospital_name', 'license_number','last_updated_on']].drop_duplicates()
h_dup

,hospital_id,hospital_name,license_number,last_updated_on
0,62915ae8-8d64-4e2f-b05f-b18edde57a3d,montefiore medical center,13-1740114,2024-07-01
169,5954cbad-a7c5-43f7-b356-8f2ecdad579a,the mount sinai hospital,330024,2024-09-16
244,40e6a8c8-a68c-4d28-b1d5-fa70d6d09636,nyu langone,7002053h,2025-01-01


In [15]:
dfh = dfh.drop(['source_file_name', 'last_updated_on',# drop this column since it only has file name coming from 3 hospitals
               'description'] # description of billing code
               ,axis=1) 

In [16]:
dfh.shape

(2947, 17)

In [17]:
# hospital dataset: checking categorical columns with less than 6 values
for col in list(dfh.columns[dfh.nunique()< 6]):
    print(col)
    print(dfh[col].nunique())
    print(dfh[col].unique())
    print(dfh[col].value_counts())
    print()

hospital_id
3
['62915ae8-8d64-4e2f-b05f-b18edde57a3d'
 '5954cbad-a7c5-43f7-b356-8f2ecdad579a'
 '40e6a8c8-a68c-4d28-b1d5-fa70d6d09636']
hospital_id
40e6a8c8-a68c-4d28-b1d5-fa70d6d09636    2322
62915ae8-8d64-4e2f-b05f-b18edde57a3d     377
5954cbad-a7c5-43f7-b356-8f2ecdad579a     248
Name: count, dtype: int64

hospital_name
3
['montefiore medical center' 'the mount sinai hospital' 'nyu langone']
hospital_name
nyu langone                  2322
montefiore medical center     377
the mount sinai hospital      248
Name: count, dtype: int64

license_number
3
['13-1740114' '330024' '7002053h']
license_number
7002053h      2322
13-1740114     377
330024         248
Name: count, dtype: int64

code_type
3
['cpt' 'ms-drg' 'local']
code_type
cpt       2159
local      410
ms-drg     378
Name: count, dtype: int64

raw_code
4
['99283' '43239' '872' 'ms-drg 872']
raw_code
43239         1524
99283         1045
ms-drg 872     290
872             88
Name: count, dtype: int64

setting
3
['outpatient' 'both' 

* hospital dataset: all **hospital_id, hospital_name, license_number** refer to different ways of identifying hospitals.

* Confiriming all there columns are different ways of referring to hospital info.

In [18]:
# payer dataset: checking categorical columns with less than 6 values 
for col in list(dfp.columns[dfp.nunique()< 6]):
    print(col)
    print(dfp[col].nunique())
    print(dfp[col].unique())
    print(dfp[col].value_counts())
    print()

payer
3
['unitedhealthcare' 'aetna' 'cigna-corporation']
payer
aetna                133
unitedhealthcare      45
cigna-corporation     44
Name: count, dtype: int64

network_name
3
['choice-plus' 'open-access-managed-choice' 'national-oap']
network_name
open-access-managed-choice    133
choice-plus                    45
national-oap                   44
Name: count, dtype: int64

network_id
3
['592bc118-0dac-4f38-949c-11dc9b3a3879'
 '39f0d406-b5df-4046-9759-f08565e45db7'
 '5dbd8f1c-3f56-4806-917b-e495668bf2bf']
network_id
39f0d406-b5df-4046-9759-f08565e45db7    133
592bc118-0dac-4f38-949c-11dc9b3a3879     45
5dbd8f1c-3f56-4806-917b-e495668bf2bf     44
Name: count, dtype: int64

code
3
[  872 99283 43239]
code
43239    103
99283    100
872       19
Name: count, dtype: int64

code_type
2
['ms-drg' 'cpt']
code_type
cpt       203
ms-drg     19
Name: count, dtype: int64

ein
3
[131740114 131624096 133971298]
ein
131740114    118
133971298     55
131624096     49
Name: count, dtype: int64

mo

* payer dataset: all **payer,network_name, network_id** refer to different ways of identifying hospitals.

In [19]:
p_dup = dfp[['payer','network_name', 'network_id']].drop_duplicates()
p_dup

,payer,network_name,network_id
0,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879
23,aetna,open-access-managed-choice,39f0d406-b5df-4046-9759-f08565e45db7
94,cigna-corporation,national-oap,5dbd8f1c-3f56-4806-917b-e495668bf2bf


In [20]:
dfp.modifier_list.value_counts()

modifier_list
gt, 95    5
95, gt    2
Name: count, dtype: int64

In [21]:
# assuming the order of items in modifier_list doesn't matter
dfp.modifier_list = dfp.modifier_list.replace('95, gt', 'gt, 95')

According to https://www.novitas-solutions.com/webcenter/portal/MedicareJH/pagebyid?contentId=00144501, they are both versions of telehealth services.

### Possible common columns for merging two datasets

In [22]:
#1
dfh.payer_name.unique() , dfp.payer.unique()

(array(['aetna', 'healthfirst', 'cigna', 'oscar', 'healthcare', 'united',
        'emblem', 'humana', 'fidelis', 'centerlight', 'partners',
        'northwell', 'agewell', 'metroplus', 'longevity', 'mvp',
        'hamaspik', 'wellcare', 'empire', 'elderplan', 'vnsny', 'nat',
        'centersplan', 'uhc', 'villagecaremax', 'senior', 'nippon',
        'seiu1199', 'molina', 'multiplan', 'magnacare', 'centivo',
        'beacon', 'united healthcare', 'bright health', 'horizon',
        'amida care', 'oxford', 'empire medicare advantage', 'lifetrac',
        'independence care', 'affinity', '1199', 'bcbs', 'hip', 'medicare',
        'somos', 'healthplus', 'ghi', 'amida', 'firsthealth', 'medicaid',
        'threerivers', 'hamaspikchoice', 'amidacare', 'archcare',
        'christian', 'american', 'brighton health', 'empire healthplus',
        'vns'], dtype=object),
 array(['unitedhealthcare', 'aetna', 'cigna-corporation'], dtype=object))

* payer_name needs be cleaned and filtered based on 3 major payer entities.

In [23]:
mapping_payer_names = {
    'united': 'unitedhealthcare',
    'uhc': 'unitedhealthcare',
    'united healthcare': 'unitedhealthcare',
    'oxford': 'unitedhealthcare',  # Oxford is a subsidiary of UnitedHealthcare
    'cigna': 'cigna-corporation'
}

# Replace values in the 'payer_name' column
dfh.payer_name = dfh.payer_name.replace(mapping_payer_names)

In [24]:
#2
dfh.code_type.unique(), dfp.code_type.unique() 

(array(['cpt', 'ms-drg', 'local'], dtype=object),
 array(['ms-drg', 'cpt'], dtype=object))

In [25]:
# 3
dfh.plan_name.nunique(), dfp.network_name.nunique()

(538, 3)

* I don't think there's two columns, plan_name and network_names are comparable.

In [26]:
# 4
dfh.raw_code.unique(), dfp.code.unique() 

(array(['99283', '43239', '872', 'ms-drg 872'], dtype=object),
 array([  872, 99283, 43239]))

In [27]:
# assumption: both raw_code from hospital and code from payer datasets are the same
# Also: ms-dr and 872 are the same
dfh.raw_code.value_counts()/len(dfh)

raw_code
43239         0.517136
99283         0.354598
ms-drg 872    0.098405
872           0.029861
Name: count, dtype: float64

In [28]:
dfh.raw_code = dfh.raw_code.replace('ms-drg 872', '872')
dfh.raw_code.value_counts()/len(dfh)

raw_code
43239    0.517136
99283    0.354598
872      0.128266
Name: count, dtype: float64

In [29]:
dfh.raw_code = dfh.raw_code.astype(int)

In [30]:
#5
dfh.hospital_id.unique(),dfp.network_id.unique()

(array(['62915ae8-8d64-4e2f-b05f-b18edde57a3d',
        '5954cbad-a7c5-43f7-b356-8f2ecdad579a',
        '40e6a8c8-a68c-4d28-b1d5-fa70d6d09636'], dtype=object),
 array(['592bc118-0dac-4f38-949c-11dc9b3a3879',
        '39f0d406-b5df-4046-9759-f08565e45db7',
        '5dbd8f1c-3f56-4806-917b-e495668bf2bf'], dtype=object))

In [31]:
# Convert columns to sets
set1 = set(dfh.hospital_id)
set2 = set(dfp.network_id)

# Find the overlap (intersection)
overlap = set1.intersection(set2)
overlap

set()

In [32]:
#6
dfh.license_number.unique(), dfp.ein.unique() # as hospital identifiers

(array(['13-1740114', '330024', '7002053h'], dtype=object),
 array([131740114, 131624096, 133971298]))

* seems there is some commonality between '13-1740114' with 131740114

In [33]:
dfh.license_number.value_counts()/len(dfh)

license_number
7002053h      0.787920
13-1740114    0.127927
330024        0.084153
Name: count, dtype: float64

In [34]:
dfp.ein.value_counts()/len(dfp)

ein
131740114    0.531532
133971298    0.247748
131624096    0.220721
Name: count, dtype: float64

In [35]:
dfp.ein = dfp.ein.astype(str)
dfp.ein = dfp.ein.replace('131740114', '13-1740114')

In [36]:
dfh.license_number.value_counts()/len(dfh)

license_number
7002053h      0.787920
13-1740114    0.127927
330024        0.084153
Name: count, dtype: float64

In [37]:
# shape before merging 
dfh.shape, dfp.shape

((2947, 17), (222, 14))

In [38]:
# merging using payer name, code type, billing code and hospital id
dfm = pd.merge(dfh, dfp, left_on=['payer_name', 'code_type','raw_code','license_number'], right_on=['payer', 'code_type','code','ein'], how='inner')

dfm.shape

(842, 30)

In [39]:
dfm= dfm.drop(['hospital_id', 'payer','raw_code','license_number'], axis=1) # we have other identifiers for hospital, payer, code conveying same info

In [40]:
d = dfm[['network_name','payer_name', 'network_id' ]]
d.drop_duplicates()

,network_name,payer_name,network_id
0,open-access-managed-choice,aetna,39f0d406-b5df-4046-9759-f08565e45db7
82,national-oap,cigna-corporation,5dbd8f1c-3f56-4806-917b-e495668bf2bf
94,choice-plus,unitedhealthcare,592bc118-0dac-4f38-949c-11dc9b3a3879


In [41]:
dfm.payer_name.value_counts()/len(dfm)    

payer_name
aetna                0.779097
unitedhealthcare     0.141330
cigna-corporation    0.079572
Name: count, dtype: float64

In [42]:
dfm = dfm.drop(['network_name', 'network_id'],axis=1) # same info about pay with different version of identifiers 

In [43]:
dfm.hospital_name.value_counts()/len(dfm)

hospital_name
montefiore medical center    1.0
Name: count, dtype: float64

In [44]:
dfm.code.value_counts()/len(dfm)

code
43239    0.557007
99283    0.437055
872      0.005938
Name: count, dtype: float64

In [45]:
dfm.hospital_name.value_counts()

hospital_name
montefiore medical center    842
Name: count, dtype: int64

In [46]:
dfm.shape

(842, 24)

In [47]:
dfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 24 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   hospital_name                          842 non-null    object 
 1   payer_name                             842 non-null    object 
 2   plan_name                              842 non-null    object 
 3   code_type                              842 non-null    object 
 4   setting                                842 non-null    object 
 5   standard_charge_gross                  421 non-null    float64
 6   standard_charge_discounted_cash        421 non-null    float64
 7   standard_charge_negotiated_dollar      842 non-null    float64
 8   standard_charge_negotiated_percentage  0 non-null      float64
 9   standard_charge_min                    842 non-null    float64
 10  standard_charge_max                    842 non-null    float64
 11  standa

* The merge dataset consists of payer_name (aetna, united healthcare, cigna), Montefiore Medical Center, billing code (43239, 99283,872) with 842 rows.

In [48]:
dfm['code'] = dfm['code'].astype(int).astype(str)
dfm['payer_code'] = dfm['payer_name'] + '_'+ dfm['code'] 

dfm = dfm.drop(['hospital_name',  'payer_name', 'code'],axis=1) # Montefiore Medical Center is the only hospital

In [49]:
dfm['payer_code'].nunique() , dfm['payer_code'].value_counts()/len(dfm)

(8,
 payer_code
 aetna_43239                0.438242
 aetna_99283                0.340855
 unitedhealthcare_43239     0.083135
 unitedhealthcare_99283     0.053444
 cigna-corporation_99283    0.042755
 cigna-corporation_43239    0.035629
 unitedhealthcare_872       0.004751
 cigna-corporation_872      0.001188
 Name: count, dtype: float64)

These columns — `standard_charge_negotiated_dollar`, `standard_charge_min`, `standard_charge_max`, `rate`, and `cms_baseline_rate` — represent different aspects of pricing in healthcare.<br>
	•	Below are some assumptions about their definitions:<br>
	•	`standard_charge_negotiated_dollar`: Represents the negotiated rate paid by the payer to the hospital for a specific service when a patient covered under their plan receives care.<br>
	•	`standard_charge_min` and `standard_charge_max`: Indicate the minimum and maximum charges that a hospital has negotiated across all payers for a specific service. These values are “de-identified,” meaning they do not disclose which payer is associated with the rates.<br>
	•	`rate`: Refers to the final amount paid by the payer (insurance company or CMS) to the hospital after applying discounts, adjustments, or other factors.<br>
	•	`cms_baseline_rate`: Represents the payment amount set by the Centers for Medicare & Medicaid Services (CMS) based on baseline rates derived from the Medicare fee schedule or other standardized payment methodologies.

* Let's filter down more columns by checking whether they get 1 value within hospital-payer-code combo.

In [50]:
# save selected str columns which may take various values within same payer_code
df_c =dfm[['payer_code','plan_name', 'code_type', 'setting',
       'standard_charge_methodology','ein','negotiation_type','billing_class']].copy()

# checking for frequency of within 'hospital_payer_code'

for i in ['plan_name', 'code_type', 'setting',
       'standard_charge_methodology','ein','negotiation_type','billing_class']:
    df_c[i + '_value_changed']= df_c.groupby(['payer_code'])[i].transform('nunique') > 1

df_c.columns

Index(['payer_code', 'plan_name', 'code_type', 'setting',
       'standard_charge_methodology', 'ein', 'negotiation_type',
       'billing_class', 'plan_name_value_changed', 'code_type_value_changed',
       'setting_value_changed', 'standard_charge_methodology_value_changed',
       'ein_value_changed', 'negotiation_type_value_changed',
       'billing_class_value_changed'],
      dtype='object')

In [51]:
df_c[['plan_name_value_changed',
       'code_type_value_changed', 'setting_value_changed',
       'standard_charge_methodology_value_changed', 'ein_value_changed',
       'negotiation_type_value_changed', 'billing_class_value_changed']].describe()

,plan_name_value_changed,code_type_value_changed,setting_value_changed,standard_charge_methodology_value_changed,ein_value_changed,negotiation_type_value_changed,billing_class_value_changed
count,842,842,842,842,842,842,842
unique,2,1,2,2,1,2,2
top,True,False,True,True,False,True,True
freq,841,842,722,435,842,656,475


* Based on this table, most of the `payer_code` entries have two distinct values for `plan_name`, `standard_charge_methodology`, `setting`, `negotiation_type`, and `billing_class`, which can contribute to differences in the rates.<br>
* The majority of `plan_name` combinations, however, have a single value for `plan_name`. For simplicity, I dropped this column since 841 out of 842 rows contain only one unique value.

In [52]:
# drop code_type, license_number since they only take 1 value within same hospital_payer_code
dfm = dfm.drop(['code_type','ein','plan_name'],axis=1)
dfm.shape

(842, 19)

In [53]:
dfm[['additional_generic_notes','standard_charge_negotiated_percentage','modifier_list']].isna().sum()/len(dfm)

additional_generic_notes                 0.985748
standard_charge_negotiated_percentage    1.000000
modifier_list                            0.983373
dtype: float64

In [54]:
# dropping these columns because they are a large number of missing values
dfm = dfm.drop(['additional_generic_notes','standard_charge_negotiated_percentage','modifier_list'],axis=1)
dfm.shape

(842, 16)

In [55]:
len(dfm[dfm.duplicated()])

48

In [56]:
# having a count for values in npi_list and place_of_service_list is easier to work with compared to the list itself
dfm['count_npi'] = dfm['taxonomy_filtered_npi_list'].apply(lambda x: len(str(x).split(',')) if pd.notnull(x) else 0)
dfm['count_place_service'] = dfm['place_of_service_list'].apply(lambda x: len(str(x).split(',')) if pd.notnull(x) else 0)

dfm = dfm.drop(['taxonomy_filtered_npi_list','place_of_service_list'], axis=1)

`place_of_service_list` according to https://www.cms.gov/medicare/coding-billing/place-of-service-codes/code-sets service codes and descriptions which is used on professional claims to specify the entity where services were rendered, e.g. 01 means Pharmacy.

`taxonomy_filtered_npi_list`: NPI (National Provider Identifier) lists, off-campus outpatient offices and main hospital campuses for billing purposes.

In [57]:
len(dfm[dfm.duplicated()])

55

In [58]:
dfm = dfm.drop_duplicates()
dfm.shape

(787, 16)

In [59]:
dfm.columns

Index(['setting', 'standard_charge_gross', 'standard_charge_discounted_cash',
       'standard_charge_negotiated_dollar', 'standard_charge_min',
       'standard_charge_max', 'standard_charge_methodology',
       'additional_payer_notes', 'billing_class', 'negotiation_type', 'rate',
       'cms_baseline_schedule', 'cms_baseline_rate', 'payer_code', 'count_npi',
       'count_place_service'],
      dtype='object')

In [60]:
dfm = dfm[['payer_code', 'setting','count_npi','count_place_service','standard_charge_gross', 'standard_charge_discounted_cash',
       'standard_charge_negotiated_dollar', 'standard_charge_min',
       'standard_charge_max', 'standard_charge_methodology',
       'additional_payer_notes', 'billing_class', 'negotiation_type', 'rate',
       'cms_baseline_schedule', 'cms_baseline_rate']]

In [61]:
dfm.head()

,payer_code,setting,count_npi,count_place_service,standard_charge_gross,standard_charge_discounted_cash,standard_charge_negotiated_dollar,standard_charge_min,standard_charge_max,standard_charge_methodology,additional_payer_notes,billing_class,negotiation_type,rate,cms_baseline_schedule,cms_baseline_rate
0,aetna_99283,outpatient,2,49,NaN,NaN,323.34,83.78,1009.22,fee schedule,NaN,professional,negotiated,46.39,pfs_nonfacility_1240201,73.68
1,aetna_99283,outpatient,2,49,NaN,NaN,323.34,83.78,1009.22,fee schedule,NaN,professional,negotiated,298.06,pfs_nonfacility_1240201,73.68
2,aetna_99283,outpatient,2,49,NaN,NaN,323.34,83.78,1009.22,fee schedule,NaN,professional,negotiated,67.85,pfs_nonfacility_1320202,78.59
3,aetna_99283,outpatient,2706,49,NaN,NaN,323.34,83.78,1009.22,fee schedule,NaN,professional,negotiated,298.06,pfs_nonfacility_1320202,78.59
4,aetna_99283,outpatient,1,49,NaN,NaN,323.34,83.78,1009.22,fee schedule,NaN,professional,negotiated,57.96,pfs_nonfacility_1320202,78.59


In [62]:
dfm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 787 entries, 0 to 828
Data columns (total 16 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   payer_code                         787 non-null    object 
 1   setting                            787 non-null    object 
 2   count_npi                          787 non-null    int64  
 3   count_place_service                787 non-null    int64  
 4   standard_charge_gross              372 non-null    float64
 5   standard_charge_discounted_cash    372 non-null    float64
 6   standard_charge_negotiated_dollar  787 non-null    float64
 7   standard_charge_min                787 non-null    float64
 8   standard_charge_max                787 non-null    float64
 9   standard_charge_methodology        787 non-null    object 
 10  additional_payer_notes             0 non-null      object 
 11  billing_class                      787 non-null    object 
 12 

In [63]:
# dropping because of missing values

dfm = dfm.drop(['standard_charge_gross','standard_charge_discounted_cash','additional_payer_notes'], axis=1)
len(dfm[dfm.duplicated()])

116

In [64]:
dfm = dfm.drop_duplicates()
dfm.shape

(671, 13)

In [65]:
dfm.payer_code.value_counts()

payer_code
aetna_43239                287
aetna_99283                280
unitedhealthcare_99283      36
unitedhealthcare_43239      28
cigna-corporation_99283     24
cigna-corporation_43239     12
unitedhealthcare_872         3
cigna-corporation_872        1
Name: count, dtype: int64

### Example 1:  United healthcare payer for 872 code (SEPTICEMIA)::

In [66]:
e1 = dfm[dfm.payer_code=='unitedhealthcare_872']

e1

,payer_code,setting,count_npi,count_place_service,standard_charge_negotiated_dollar,standard_charge_min,standard_charge_max,standard_charge_methodology,billing_class,negotiation_type,rate,cms_baseline_schedule,cms_baseline_rate
183,unitedhealthcare_872,inpatient,23,0,57020.41,4296.35,57084.61,fee schedule,institutional,negotiated,15902.0,ipps,6829.75
744,unitedhealthcare_872,inpatient,23,0,13570.00,4296.35,57084.61,fee schedule,institutional,negotiated,15902.0,ipps,6829.75
745,unitedhealthcare_872,inpatient,23,0,53002.77,4296.35,57084.61,fee schedule,institutional,negotiated,15902.0,ipps,6829.75


In [67]:
e1 = e1.loc[:, e1.nunique() > 1]
e1

,standard_charge_negotiated_dollar
183,57020.41
744,13570.00
745,53002.77


For this billing code 872 (SEPTICEMIA OR SEVERE SEPSIS WITHOUT MV >96 HOURS WITHOUT MCC) with cigna payer, we can see:<br>
* same `rate` with varying `standard_charge_negotiated_dollar`.
* `rate` is more than `cms_baseline_rate` and `standard_charge_min` but less than `standard_charge_negotiated_dollar` and `standard_charge_max`.
* I'd pick `rate` 15,902 for SEPTICEMIA paid by UHC.

### Example 2: Cigna corporation payer for 43239 code (Esophagogastroduodenoscopy (EGD) with biopsy):

In [68]:
e2 = dfm[dfm.payer_code=='cigna-corporation_43239']
print(e2.shape)
e2

(12, 13)


,payer_code,setting,count_npi,count_place_service,standard_charge_negotiated_dollar,standard_charge_min,standard_charge_max,standard_charge_methodology,billing_class,negotiation_type,rate,cms_baseline_schedule,cms_baseline_rate
314,cigna-corporation_43239,both,836,50,841.81,146.49,1733.49,fee schedule,professional,fee schedule,841.81,pfs_nonfacility_1320202,424.76
315,cigna-corporation_43239,both,5,50,841.81,146.49,1733.49,fee schedule,professional,fee schedule,841.81,pfs_nonfacility_1320202,424.76
316,cigna-corporation_43239,both,1,50,841.81,146.49,1733.49,fee schedule,professional,fee schedule,395.58,pfs_nonfacility_1320202,424.76
317,cigna-corporation_43239,both,3,20,841.81,146.49,1733.49,fee schedule,professional,fee schedule,170.21,pfs_facility_1320201,151.42
318,cigna-corporation_43239,both,3,28,841.81,146.49,1733.49,fee schedule,professional,fee schedule,509.63,pfs_nonfacility_1320201,413.17
319,cigna-corporation_43239,both,28,50,841.81,146.49,1733.49,fee schedule,professional,fee schedule,708.10,pfs_nonfacility_1320202,424.76
691,cigna-corporation_43239,outpatient,836,50,5954.00,2518.00,9347.00,case rate,professional,fee schedule,841.81,pfs_nonfacility_1320202,424.76
692,cigna-corporation_43239,outpatient,5,50,5954.00,2518.00,9347.00,case rate,professional,fee schedule,841.81,pfs_nonfacility_1320202,424.76
693,cigna-corporation_43239,outpatient,1,50,5954.00,2518.00,9347.00,case rate,professional,fee schedule,395.58,pfs_nonfacility_1320202,424.76
694,cigna-corporation_43239,outpatient,3,20,5954.00,2518.00,9347.00,case rate,professional,fee schedule,170.21,pfs_facility_1320201,151.42


In [69]:
#keeping columns with different values 
e2 = e2.loc[:, e2.nunique() > 1] 
e2.columns

Index(['setting', 'count_npi', 'count_place_service',
       'standard_charge_negotiated_dollar', 'standard_charge_min',
       'standard_charge_max', 'standard_charge_methodology', 'rate',
       'cms_baseline_schedule', 'cms_baseline_rate'],
      dtype='object')

* In this case, I filter the rows based on either `setting` or `standard_charge_methodology` beacsue either does the same filetring.

In [70]:
e2

,setting,count_npi,count_place_service,standard_charge_negotiated_dollar,standard_charge_min,standard_charge_max,standard_charge_methodology,rate,cms_baseline_schedule,cms_baseline_rate
314,both,836,50,841.81,146.49,1733.49,fee schedule,841.81,pfs_nonfacility_1320202,424.76
315,both,5,50,841.81,146.49,1733.49,fee schedule,841.81,pfs_nonfacility_1320202,424.76
316,both,1,50,841.81,146.49,1733.49,fee schedule,395.58,pfs_nonfacility_1320202,424.76
317,both,3,20,841.81,146.49,1733.49,fee schedule,170.21,pfs_facility_1320201,151.42
318,both,3,28,841.81,146.49,1733.49,fee schedule,509.63,pfs_nonfacility_1320201,413.17
319,both,28,50,841.81,146.49,1733.49,fee schedule,708.10,pfs_nonfacility_1320202,424.76
691,outpatient,836,50,5954.00,2518.00,9347.00,case rate,841.81,pfs_nonfacility_1320202,424.76
692,outpatient,5,50,5954.00,2518.00,9347.00,case rate,841.81,pfs_nonfacility_1320202,424.76
693,outpatient,1,50,5954.00,2518.00,9347.00,case rate,395.58,pfs_nonfacility_1320202,424.76
694,outpatient,3,20,5954.00,2518.00,9347.00,case rate,170.21,pfs_facility_1320201,151.42


In [71]:
e21 = e2[e2.setting=='both']
e22 = e2[e2.setting!='both']

#keeping columns with different values 
e21 = e21.loc[:, e21.nunique() > 1] 
e22 = e22.loc[:, e22.nunique() > 1]

In [72]:
# I'd just focus on e21, since same process could be implemented to e22
e21

,count_npi,count_place_service,rate,cms_baseline_schedule,cms_baseline_rate
314,836,50,841.81,pfs_nonfacility_1320202,424.76
315,5,50,841.81,pfs_nonfacility_1320202,424.76
316,1,50,395.58,pfs_nonfacility_1320202,424.76
317,3,20,170.21,pfs_facility_1320201,151.42
318,3,28,509.63,pfs_nonfacility_1320201,413.17
319,28,50,708.10,pfs_nonfacility_1320202,424.76


In [73]:
# filter based on count nPI and count place service
e21['max_npi_count'] = e21.groupby('cms_baseline_schedule')['count_npi'].transform('max')
e21['max_count_place_service'] = e21.groupby('cms_baseline_schedule')['count_place_service'].transform('max')
e21= e21[e21.count_npi==e21.max_npi_count].drop('max_npi_count',axis=1)
e21= e21[e21.count_place_service==e21.max_count_place_service].drop('max_count_place_service',axis=1)
e21.sort_values('cms_baseline_schedule', inplace=True)
e21

,count_npi,count_place_service,rate,cms_baseline_schedule,cms_baseline_rate
317,3,20,170.21,pfs_facility_1320201,151.42
318,3,28,509.63,pfs_nonfacility_1320201,413.17
314,836,50,841.81,pfs_nonfacility_1320202,424.76


I pick `rate` column as the final price where different rates are implemented for PFS (Physician Fee Schedule) facility and non-facility refer to the facility-based payment and  non-facility-based rates under the Medicare Physician Fee Schedule (PFS).<br>
For PFS Facility, these rates are applied when services are provided in facility settings such as hospitals, skilled nursing facilities or ambulatory surgical centers. For PFS non-facility, these rates apply to services provided in non-facility settings, such as a physician’s office or a patient’s home, where the physician bears the cost of resources like labor, medical supplies, and equipment.

-----------

The ultimate goal for each payer_code combination is to:<br>
	1.	Filter out redundant columns: Eliminate as many columns as possible, especially those that are exact duplicates or provide no additional value.<br>
	2.	Identify the largest values for `count_place_service` and `count_npi` within certain categories.<br>
	3.	Resolve remaining rate differences:<br>
	•	If there are still differences in the `'rate'`, use additional columns such as `setting`,`cms_baseline_schedule` and `standard_charge_methodology` to further narrow down the choices.